# Poisson Example 2D

Authors: Kidus Teshome, Cameron Seebeck, Cian Wilson

## Description

As a reminder, in this case we are seeking the approximate solution to
\begin{equation}
- \nabla^2 T = -\tfrac{5}{4} \exp \left( x+\tfrac{y}{2} \right)
\end{equation}
in a unit square, $\Omega=[0,1]\times[0,1]$, imposing the boundary conditions
\begin{align}
  T &= \exp\left(x+\tfrac{y}{2}\right) && \text{on } \partial\Omega \text{ where } x=0 \text{ or } y=0 \\
  \nabla T\cdot \hat{\vec{n}} &= \exp\left(x + \tfrac{y}{2}\right) && \text{on } \partial\Omega \text{ where } x=1  \\
  \nabla T\cdot \hat{\vec{n}} &= \tfrac{1}{2}\exp\left(x + \tfrac{y}{2}\right) && \text{on } \partial\Omega \text{ where } y=1
 \end{align}

The analytical solution to this problem is $T(x,y) = \exp\left(x+\tfrac{y}{2}\right)$.

## Themes and variations

* Given that we know the exact solution to this problem is $T(x,y)$=$\exp\left(x+\tfrac{y}{2}\right)$ write a python function to evaluate the error in our numerical solution.
* Loop over a variety of numbers of elements, `ne`, and polynomial degrees, `p`, and check that the numerical solution converges with an increasing number of degrees of freedom.
* Write an equation for the gradient of $\tilde{T}$, describe it using UFL, solve it, and plot the solution.

### Preamble

Start by loading `solve_poisson_2d` from `notebooks/poisson_2d.ipynb` and setting up some paths.

In [ ]:
import ipyparallel as ipp
np = 2
rc = ipp.Cluster(engine_launcher_class="mpi", n=np).start_and_connect_sync()

Starting 2 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/2 [00:00<?, ?engine/s]

mpiexec error output:
-------------------------------------------------------------------------------------------------------
Build dofmap data                                                           |     2  0.023841  0.047682
Build sparsity                                                              |     1  0.077160  0.077160
Compute connectivity 1-0                                                    |     1  0.003491  0.003491
Compute dof reordering map                                                  |     2  0.002667  0.005333
Compute entities of dim = 1                                                 |     1  0.050300  0.050300
Compute graph partition (SCOTCH)                                            |     1  0.088290  0.088290
Compute local part of mesh dual graph                                       |     2  0.116206  0.232412
Compute local-to-local map                                                  |     1  0.018157  0.018157
Compute non-local part of mesh dual graph 

In [2]:
%%px
from poisson_2d import solve_poisson_2d
from mpi4py import MPI
import numpy as np
import ufl
import sys, os
basedir = ''
if "__file__" in globals(): basedir = os.path.dirname(__file__)
sys.path.append(os.path.join(basedir, os.path.pardir, 'python'))
import utils
import matplotlib.pyplot as pl
import pyvista as pv
if __name__ == "__main__" and "__file__" in globals():
    pv.OFF_SCREEN = True
import pathlib
if __name__ == "__main__":
    output_folder = pathlib.Path(os.path.join(basedir, "output"))
    output_folder.mkdir(exist_ok=True, parents=True)
import time

%px:   0%|          | 0/2 [00:00<?, ?tasks/s]

In [3]:
%%px
print(MPI.COMM_WORLD.rank, MPI.COMM_WORLD.size)

[stdout:1] 1 2


[stdout:0] 0 2


In [4]:
%%px
if __name__ == "__main__":
    ne = 640
    p = 2
    # Solve the 2D Poisson problem
    start_time = time.time()
    T_i = solve_poisson_2d(ne, p)
    end_time = time.time()
    comm = T_i.function_space.mesh.comm
    print(f"{comm.rank} ({T_i.function_space.dofmap.index_map.size_local}) - time taken: {end_time - start_time} seconds")

[stdout:0] Generate mesh (0) 0.66274094581604
Generate functionspace (0) 0.0859217643737793
Generate functions (0) 6.556510925292969e-05
Generate Dirichlet bcs (0) 0.071990966796875
Generate Neumann bcs (0) 0.000469207763671875
Generate forms (0) 0.005739450454711914
Assemble problem (0) 0.41512322425842285
Solve problem (0) 11.12532353401184
0 (820490) - time taken: 12.367806673049927 seconds

[MPI_MAX] Summary of timings                                                |  reps  wall avg  wall tot
-------------------------------------------------------------------------------------------------------
Build dofmap data                                                           |     2  0.023841  0.047682
Build sparsity                                                              |     1  0.077160  0.077160
Compute connectivity 1-0                                                    |     1  0.003491  0.003491
Compute dof reordering map                                                  |     

[stdout:1] Generate mesh (1) 0.6630446910858154
Generate functionspace (1) 0.08593606948852539
Generate functions (1) 5.507469177246094e-05
Generate Dirichlet bcs (1) 0.0711526870727539
Generate Neumann bcs (1) 0.0005049705505371094
Generate forms (1) 0.007612705230712891
Assemble problem (1) 0.41404247283935547
Solve problem (1) 11.125169515609741
1 (820471) - time taken: 12.36797022819519 seconds


%px:   0%|          | 0/2 [00:00<?, ?tasks/s]

In [5]:
%%px
if __name__ == "__main__":
    # plot the solution as a colormap
    plotter = utils.plot_scalar(T_i, gather=True)
    # save the plot
    utils.plot_save(plotter, output_folder / "2d_poisson_test_single_solution_ipyparallel_ne{:d}.png".format(ne,))
    comm = T_i.function_space.mesh.comm
    if comm.size > 1:
        # if we're running in parallel (e.g. from a script) then save an image per process as well
        plotter_p = utils.plot_scalar(T_i)
        utils.plot_save(plotter_p, output_folder / "2d_poisson_test_single_solution_ipyparallel_ne{:d}_p{:d}.png".format(ne,comm.rank,))

%px:   0%|          | 0/2 [00:00<?, ?tasks/s]